# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd
import math
import json
import requests
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
symbol = 'AAPL'
api_url =f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.08798550504620395

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(data, n):
    return [data[x:x+n] for x in range(0, len(data), n)]

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
symbol_groups = list(chunks(stocks['Ticker'],100))



Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
def chunks(data, n):
    return [data[x:x+n] for x in range(0, len(data), n)]

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
symbol_groups = list(chunks(stocks['Ticker'], 100))
df = pd.DataFrame(columns=my_columns)
processed_symbols = set()  # Keep track of processed symbols

# Fetch data for each chunk of symbols
for symbol_group in symbol_groups:
    symbol_string = ','.join(symbol_group)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'

    response = requests.get(batch_api_call_url)
    if response.status_code == 200:
        data = response.json()
        rows_to_add = [] 
        for symbol in symbol_string.split(','):
            if symbol in data and symbol not in processed_symbols:
                row_data = [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ]
                rows_to_add.append(row_data)
                processed_symbols.add(symbol)  # Add symbol to the set of processed symbols

        if rows_to_add:
            chunk_df = pd.DataFrame(rows_to_add, columns=my_columns)
            df = pd.concat([df, chunk_df], ignore_index=True)
    else:
        print(f"Error fetching data for symbols {symbol_group}: Status Code {response.status_code}")

# Now you have a DataFrame containing the stock data without duplicates
df


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,113.430,-0.188215,N/A
1,AAL,13.305,-0.024879,N/A
2,AAP,57.930,-0.672006,N/A
3,AAPL,174.200,0.087986,N/A
4,ABBV,151.160,0.091997,N/A
...,...,...,...,...
496,YUM,129.070,0.112489,N/A
497,ZBH,120.630,0.025977,N/A
498,ZBRA,250.100,-0.194132,N/A
499,ZION,36.140,-0.338243,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = df[:50]
final_df.reset_index()

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.920,2.281287,N/A
1,343,NVDA,454.900,2.095309,N/A
2,193,FTI,20.790,1.367001,N/A
3,388,RCL,97.350,1.060783,N/A
4,197,GE,113.600,0.947552,N/A
5,281,LLY,598.880,0.921770,N/A
6,368,PHM,76.960,0.883337,N/A
7,330,NFLX,412.090,0.837780,N/A
8,50,AVGO,853.440,0.652230,N/A
9,313,MPC,155.480,0.597036,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df)):
    final_df.iloc[i, final_df.columns.get_loc('Number of Shares to Buy')] = math.floor(position_size / final_df.iloc[i, final_df.columns.get_loc('Price')])

final_df.reset_index()


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.920,2.281287,2502
1,343,NVDA,454.900,2.095309,439
2,193,FTI,20.790,1.367001,9620
3,388,RCL,97.350,1.060783,2054
4,197,GE,113.600,0.947552,1760
5,281,LLY,598.880,0.921770,333
6,368,PHM,76.960,0.883337,2598
7,330,NFLX,412.090,0.837780,485
8,50,AVGO,853.440,0.652230,234
9,313,MPC,155.480,0.597036,1286


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [60]:
hqm_columns = [
    'Ticker',
    'Price', 
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
processed_symbols = set()  # Keep track of processed symbols

# Create an empty list to store DataFrames to concatenate later
dfs_to_concat = []

# Fetch data for each chunk of symbols
for symbol_group in symbol_groups:
    symbol_string = ','.join(symbol_group)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'

    response = requests.get(batch_api_call_url)
    if response.status_code == 200:
        data = response.json()
        
        # Create a list of lists to store data for each symbol
        symbol_data_list = []
        
        # Iterate through symbols in the group
        for symbol in symbol_string.split(','):
            # Check if the symbol exists in the data dictionary and has not been processed
            if symbol in data and symbol not in processed_symbols:
                # Extract relevant data from the API response
                symbol_data = data[symbol]
                
                # Calculate the number of shares to buy (you can replace 'N/A' with your calculation logic)
                number_of_shares_to_buy = 'N/A'
                
                # Append data for the current symbol to the list
                symbol_data_list.append([
                    symbol,
                    symbol_data['price'],
                    number_of_shares_to_buy,
                    symbol_data['stats']['year1ChangePercent'],
                    'N/A',  # You'll calculate percentiles later
                    symbol_data['stats']['month6ChangePercent'],
                    'N/A',
                    symbol_data['stats']['month3ChangePercent'],
                    'N/A',
                    symbol_data['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ])
                
                # Add the symbol to the set of processed symbols
                processed_symbols.add(symbol)
        
        # Create a DataFrame for the current chunk of symbols
        chunk_df = pd.DataFrame(symbol_data_list, columns=hqm_columns)
        
        # Append the chunk DataFrame to the list for concatenation
        dfs_to_concat.append(chunk_df)
    else:
        print(f"Error fetching data for symbols {symbol_group}: Status Code {response.status_code}")

# Concatenate all DataFrames in the list into a single DataFrame
hqm_dataframe = pd.concat(dfs_to_concat, ignore_index=True)

# Calculate percentiles for the return columns (you'll need to implement this part)
# You can use pandas' rank method to calculate percentiles
# For example, hqm_dataframe['One-Year_Return_Percentile'] = hqm_dataframe['One-Year Price Return'].rank(pct=True)

# Now you have a DataFrame containing the HQM data without duplicate symbols
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,113.430,N/A,-0.188215,N/A,-0.166081,N/A,-0.046775,N/A,-0.110988,N/A,N/A
1,AAL,13.305,N/A,-0.024879,N/A,-0.087322,N/A,-0.119226,N/A,-0.098403,N/A,N/A
2,AAP,57.930,N/A,-0.672006,N/A,-0.544006,N/A,-0.137931,N/A,-0.205036,N/A,N/A
3,AAPL,174.200,N/A,0.087986,N/A,0.192060,N/A,-0.038161,N/A,-0.008381,N/A,N/A
4,ABBV,151.160,N/A,0.091997,N/A,0.017388,N/A,0.095803,N/A,-0.019122,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.070,N/A,0.112489,N/A,0.037569,N/A,-0.044344,N/A,-0.041282,N/A,N/A
497,ZBH,120.630,N/A,0.025977,N/A,-0.013486,N/A,-0.102679,N/A,-0.020173,N/A,N/A
498,ZBRA,250.100,N/A,-0.194132,N/A,-0.127971,N/A,-0.102432,N/A,-0.040582,N/A,N/A
499,ZION,36.140,N/A,-0.338243,N/A,-0.053795,N/A,0.240759,N/A,-0.020351,N/A,N/A


In [61]:
# Calculate medians for the return columns
median_one_year_return = hqm_dataframe['One-Year Price Return'].median()
median_six_month_return = hqm_dataframe['Six-Month Price Return'].median()
median_three_month_return = hqm_dataframe['Three-Month Price Return'].median()
median_one_month_return = hqm_dataframe['One-Month Price Return'].median()

# Fill missing values with their respective medians
hqm_dataframe['One-Year Price Return'].fillna(median_one_year_return, inplace=True)
hqm_dataframe['Six-Month Price Return'].fillna(median_six_month_return, inplace=True)
hqm_dataframe['Three-Month Price Return'].fillna(median_three_month_return, inplace=True)
hqm_dataframe['One-Month Price Return'].fillna(median_one_month_return, inplace=True)


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [62]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        price_return_col = f'{time_period} Price Return'
        price_return_value = hqm_dataframe.loc[row, price_return_col]        
        percentile = score(hqm_dataframe[price_return_col], price_return_value)/100
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = percentile

# Print the DataFrame to check the results
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,113.430,N/A,-0.188215,0.149701,-0.166081,0.075848,-0.046775,0.265469,-0.110988,0.053892,N/A
1,AAL,13.305,N/A,-0.024879,0.389222,-0.087322,0.157685,-0.119226,0.085828,-0.098403,0.085828,N/A
2,AAP,57.930,N/A,-0.672006,0.005988,-0.544006,0.003992,-0.137931,0.065868,-0.205036,0.013972,N/A
3,AAPL,174.200,N/A,0.087986,0.606786,0.192060,0.774451,-0.038161,0.299401,-0.008381,0.606786,N/A
4,ABBV,151.160,N/A,0.091997,0.61477,0.017388,0.413174,0.095803,0.770459,-0.019122,0.487026,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.070,N/A,0.112489,0.650699,0.037569,0.463074,-0.044344,0.279441,-0.041282,0.319361,N/A
497,ZBH,120.630,N/A,0.025977,0.522954,-0.013486,0.309381,-0.102679,0.105788,-0.020173,0.481038,N/A
498,ZBRA,250.100,N/A,-0.194132,0.137725,-0.127971,0.107784,-0.102432,0.107784,-0.040582,0.325349,N/A
499,ZION,36.140,N/A,-0.338243,0.0499,-0.053795,0.217565,0.240759,0.958084,-0.020351,0.479042,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [63]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe  
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,113.430,N/A,-0.188215,0.149701,-0.166081,0.075848,-0.046775,0.265469,-0.110988,0.053892,0.136228
1,AAL,13.305,N/A,-0.024879,0.389222,-0.087322,0.157685,-0.119226,0.085828,-0.098403,0.085828,0.179641
2,AAP,57.930,N/A,-0.672006,0.005988,-0.544006,0.003992,-0.137931,0.065868,-0.205036,0.013972,0.022455
3,AAPL,174.200,N/A,0.087986,0.606786,0.192060,0.774451,-0.038161,0.299401,-0.008381,0.606786,0.571856
4,ABBV,151.160,N/A,0.091997,0.61477,0.017388,0.413174,0.095803,0.770459,-0.019122,0.487026,0.571357
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.070,N/A,0.112489,0.650699,0.037569,0.463074,-0.044344,0.279441,-0.041282,0.319361,0.428144
497,ZBH,120.630,N/A,0.025977,0.522954,-0.013486,0.309381,-0.102679,0.105788,-0.020173,0.481038,0.35479
498,ZBRA,250.100,N/A,-0.194132,0.137725,-0.127971,0.107784,-0.102432,0.107784,-0.040582,0.325349,0.169661
499,ZION,36.140,N/A,-0.338243,0.0499,-0.053795,0.217565,0.240759,0.958084,-0.020351,0.479042,0.426148


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [64]:
hqm_dataframe.sort_values('HQM Score', ascending=False,inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop= True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,20.790,N/A,1.367001,0.996008,0.476531,0.978044,0.473493,0.996008,0.140741,0.994012,0.991018
1,LLY,598.880,N/A,0.921770,0.99002,0.911456,0.998004,0.348689,0.984032,0.136868,0.992016,0.991018
2,LB,79.920,N/A,2.281287,1.0,0.815831,0.996008,0.215421,0.946108,0.079271,0.968064,0.977545
3,NVDA,454.900,N/A,2.095309,0.998004,0.954206,1.0,0.136561,0.872255,0.098365,0.98004,0.962575
4,BKR,36.240,N/A,0.513199,0.966068,0.308416,0.928144,0.259527,0.968064,0.045660,0.912176,0.943613
5,ETN,221.410,N/A,0.573774,0.976048,0.350444,0.952096,0.207103,0.936128,0.043689,0.894212,0.939621
6,INTC,38.730,N/A,0.282686,0.884232,0.449713,0.97006,0.183555,0.916168,0.113786,0.988024,0.939621
7,ADBE,553.655,N/A,0.367974,0.92016,0.646553,0.994012,0.142385,0.882236,0.066398,0.958084,0.938623
8,PSX,123.170,N/A,0.443014,0.938124,0.252973,0.860279,0.296164,0.97006,0.075470,0.966068,0.933633
9,MPC,155.480,N/A,0.597036,0.982036,0.226681,0.832335,0.393796,0.994012,0.045882,0.914172,0.930639


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [67]:
portfolio_input()

Enter the size of your portfolio:1000000


In [68]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor( position_size/hqm_dataframe.loc[i,'Price'])
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,20.790,962,1.367001,0.996008,0.476531,0.978044,0.473493,0.996008,0.140741,0.994012,0.991018
1,LLY,598.880,33,0.921770,0.99002,0.911456,0.998004,0.348689,0.984032,0.136868,0.992016,0.991018
2,LB,79.920,250,2.281287,1.0,0.815831,0.996008,0.215421,0.946108,0.079271,0.968064,0.977545
3,NVDA,454.900,43,2.095309,0.998004,0.954206,1.0,0.136561,0.872255,0.098365,0.98004,0.962575
4,BKR,36.240,551,0.513199,0.966068,0.308416,0.928144,0.259527,0.968064,0.045660,0.912176,0.943613
5,ETN,221.410,90,0.573774,0.976048,0.350444,0.952096,0.207103,0.936128,0.043689,0.894212,0.939621
6,INTC,38.730,516,0.282686,0.884232,0.449713,0.97006,0.183555,0.916168,0.113786,0.988024,0.939621
7,ADBE,553.655,36,0.367974,0.92016,0.646553,0.994012,0.142385,0.882236,0.066398,0.958084,0.938623
8,PSX,123.170,162,0.443014,0.938124,0.252973,0.860279,0.296164,0.97006,0.075470,0.966068,0.933633
9,MPC,155.480,128,0.597036,0.982036,0.226681,0.832335,0.393796,0.994012,0.045882,0.914172,0.930639


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [69]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [70]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [71]:
column_format = {
    'A': ['Ticker', string_template] ,
    'B': ['Price', dollar_template] , 
    'C': ['Number of Shares to Buy', integer_template] ,
    'D': ['One-Year Price Return', percent_template] ,
    'E': ['One-Year Return Percentile', percent_template] ,
    'F': ['Six-Month Price Return', percent_template] ,
    'G': ['Six-Month Return Percentile', percent_template] ,
    'H': ['Three-Month Price Return', percent_template] ,
    'I': ['Three-Month Return Percentile', percent_template] ,
    'J': ['One-Month Price Return', percent_template] ,
    'K': ['One-Month Return Percentile', percent_template] ,
    'L': ['HQM Score', percent_template]
}

for col in column_format:
    writer.sheets['Momentum Strategy'].set_column(f'{col}:{col}', 22, column_format[col][1])
    writer.sheets['Momentum Strategy'].write(f'{col}1', column_format[col][0], column_format[col][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [72]:
writer.save()